In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import re

In [2]:
# Define the target genome as a fasta file
fasta_file_path = ""
output_dir = ""
target_region_start = 2542000
target_region_end = 2557445
PAM = "ttc"
PAM_rev = "gaa"
spacer_length = 34
seed_length = 8

# Read the fasta file
seq_record = SeqIO.read(fasta_file_path, "fasta")
genome = seq_record.seq
genome_seq = str(genome).lower()
genome_revcomp = genome.reverse_complement()

print("Searching for pCRISPR-Cas3 (type I-C) protospacer sequences in: {seq_record.id} (Genome Size: {len(seq_record)} bp)")
print("The following parameters will be used: PAM motif: ", PAM, ", protospacer length: ", spacer_length, ", seed sequence length: ", seed_length)

Searching for pCRISPR-Cas3 (type I-C) protospacer sequences in: {seq_record.id} (Genome Size: {len(seq_record)} bp)
The following parameters will be used: PAM motif:  ttc , protospacer length:  34 , seed sequence length:  8


In [3]:
%%time

#dictionary for the spacers
hits = {"Spacer": [], "Start": [], "End":[], "0 nt matches":[], "Seed 0 nt matches":[], "Strand":[]}
PAM_sites = 0

#Search for PAM sequences on the forward strand
p = re.compile(PAM)
iterator = p.finditer(genome_seq)

#parse through PAM hits
for match in iterator:
    #look only at the hits within the target region
    if (target_region_start < match.span()[0] < target_region_end):
        PAM_sites += 1
        spacer_seq = str(genome[(match.span()[1]):(match.span()[1]+spacer_length)])
        seed_seq = str(genome[(match.span()[0]):(match.span()[0] + (seed_length+3))])
        hits["Spacer"].append(spacer_seq)
        hits["Start"].append(match.span()[1]+1)
        hits["End"].append(match.span()[1]+spacer_length)
        #Count off targets on both strands. For Seed off targets, the PAM sequence is included; PAM+8nt
        hits["0 nt matches"].append(genome.count_overlap(spacer_seq)+ genome_revcomp.count_overlap(spacer_seq))
        hits["Seed 0 nt matches"].append(genome.count_overlap(seed_seq) + genome_revcomp.count_overlap(seed_seq))
        hits["Strand"].append("Forward")

print("Forward done")

#Search for PAM sequences on the reverse strand
q = re.compile(PAM_rev)
iterator_2 = q.finditer(genome_seq)       

#parse through PAM hits
for match in iterator_2:
    #look only at the hits within the target region
    if (target_region_start < match.span()[0] < target_region_end):
        PAM_sites += 1
        spacer_seq = genome[(match.span()[0]-spacer_length):(match.span()[0])]
        seed_seq = str(genome[(match.span()[1] - (seed_length+3)):(match.span()[1])])                                         
        hits["Spacer"].append(str(spacer_seq.reverse_complement()))
        hits["Start"].append(match.span()[0]-spacer_length)
        hits["End"].append(match.span()[0])
        #Count off targets on both strands. For Seed off targets, the PAM sequence is included; PAM+8nt
        hits["0 nt matches"].append(genome.count_overlap(spacer_seq) + genome_revcomp.count_overlap(spacer_seq))
        hits["Seed 0 nt matches"].append(genome.count_overlap(seed_seq) + genome_revcomp.count_overlap(seed_seq))
        hits["Strand"].append("Reverse")

print("Reverse done")
print("Scanned ",PAM_sites, "potential PAM sites within the target region (", target_region_start, " - ", target_region_end, ").", len(hits["Spacer"]), " potential protospacer sequences were analyzed.")

Forward done
Reverse done
Scanned  229 potential PAM sites within the target region ( 2542000  -  2557445 ). 229  potential protospacer sequences were analyzed.
CPU times: user 22.5 s, sys: 66.5 ms, total: 22.6 s
Wall time: 22.7 s


In [4]:
#Turn dictionary into dataframe
spacer_table = pd.DataFrame.from_dict(hits)

#Automatically generate primer sequences for selected spacers
#Simple addition of substrings: spacer sequence to the fwd primer, rev complement of the spacer sequence to the rev primer
spacer_table["Fwd Primer"] = spacer_table["Spacer"] + str("GTCGCcCggCaaAaccGg")
spacer_table["Rev Primer"] = spacer_table["Spacer"].map(lambda x: str(Seq(x).reverse_complement())) + str("GTTTCAATCCACGCGCCCGT")
pd.set_option('display.max_colwidth', None)

In [5]:
#return the 20 spacers with the lowest number of off target hits. There can be more than 10 spacers with no off target hits
spacer_table.nsmallest(20, "Seed 0 nt matches")

,Spacer,Start,End,0 nt matches,Seed 0 nt matches,Strand,Fwd Primer,Rev Primer
9,tccgttccggggcgtgctccgatgtaccgatgcg,2543736,2543769,1,1,Forward,tccgttccggggcgtgctccgatgtaccgatgcgGTCGCcCggCaaAaccGg,cgcatcggtacatcggagcacgccccggaacggaGTTTCAATCCACGCGCCCGT
68,cctggggcggccggggcgcgggccgggggcgtcg,2555269,2555302,1,1,Forward,cctggggcggccggggcgcgggccgggggcgtcgGTCGCcCggCaaAaccGg,cgacgcccccggcccgcgccccggccgccccaggGTTTCAATCCACGCGCCCGT
77,ggggttcaggccctggtcgagggcggcgtccgcc,2556759,2556792,1,1,Forward,ggggttcaggccctggtcgagggcggcgtccgccGTCGCcCggCaaAaccGg,ggcggacgccgccctcgaccagggcctgaaccccGTTTCAATCCACGCGCCCGT
79,ggcacgtggtcgaccggctccatgtggacccggc,2556871,2556904,1,1,Forward,ggcacgtggtcgaccggctccatgtggacccggcGTCGCcCggCaaAaccGg,gccgggtccacatggagccggtcgaccacgtgccGTTTCAATCCACGCGCCCGT
98,aggggttaccgtggcgttccgatgcgagcgcatc,2543763,2543797,1,1,Reverse,aggggttaccgtggcgttccgatgcgagcgcatcGTCGCcCggCaaAaccGg,gatgcgctcgcatcggaacgccacggtaacccctGTTTCAATCCACGCGCCCGT
195,aaggggttcttcaccgcccggttcgggcgcggcg,2553788,2553822,1,1,Reverse,aaggggttcttcaccgcccggttcgggcgcggcgGTCGCcCggCaaAaccGg,cgccgcgcccgaaccgggcggtgaagaaccccttGTTTCAATCCACGCGCCCGT
23,gtcctcaggtacggggccgtccgggcagcgtgcg,2546026,2546059,1,2,Forward,gtcctcaggtacggggccgtccgggcagcgtgcgGTCGCcCggCaaAaccGg,cgcacgctgcccggacggccccgtacctgaggacGTTTCAATCCACGCGCCCGT
24,cgctgtcaggccactgtcagccggtgacagcggg,2546088,2546121,1,2,Forward,cgctgtcaggccactgtcagccggtgacagcgggGTCGCcCggCaaAaccGg,cccgctgtcaccggctgacagtggcctgacagcgGTTTCAATCCACGCGCCCGT
61,gcccctcagctgttccaggaggagccgtctgcgg,2554033,2554066,1,2,Forward,gcccctcagctgttccaggaggagccgtctgcggGTCGCcCggCaaAaccGg,ccgcagacggctcctcctggaacagctgaggggcGTTTCAATCCACGCGCCCGT
76,tgccagtacttgatggagaggaccaggtactggt,2556373,2556406,1,2,Forward,tgccagtacttgatggagaggaccaggtactggtGTCGCcCggCaaAaccGg,accagtacctggtcctctccatcaagtactggcaGTTTCAATCCACGCGCCCGT


In [ ]:
#Export entire spacer table as csv
spacer_table.to_csv(output_dir)